In [6]:
!pip install psycopg2
!pip install SQLAlchemy
import pandas as pd
import psycopg2
import os
import matplotlib
from sqlalchemy import create_engine
from tqdm import tqdm_notebook

In [7]:
!pip install yfinance
import yfinance as yf
import json
import requests

In [8]:
db_password = "<Your Own Password>"  # Set to your own password
engine = create_engine('postgresql+psycopg2://postgres:{}@<Connection IP to GCP>/<DB name to connect to>'.format(db_password))

In [9]:
companies_info = {}

def get_company_info(ticker):

    """
    Function that gets the information of companies. 
    Information such as Symbol, Name, Sector, Exchange, Website and Zip code of the company
    """
    stock_info = yf.Ticker('{}'.format(ticker)).info
    companies_info[ticker] = {}
    companies_info[ticker]['Company_Symbol'] = stock_info['symbol']
    companies_info[ticker]['Company Name'] = stock_info['shortName']
    companies_info[ticker]['Sector'] = stock_info['sector']
    companies_info[ticker]['Exchange'] = stock_info['exchange']
    companies_info[ticker]['Website'] = stock_info['website']
    companies_info[ticker]['Zip'] = stock_info['zip']

In [10]:
top_companies = ['AAPL','MSFT','GOOGL','SAR','AMZN','TSLA','FB','NVDA','TSM']
for ticker in top_companies:
  get_company_info(ticker)

In [11]:
company_info = pd.DataFrame.from_dict(companies_info, orient='index')
company_info

,Company_Symbol,Company Name,Sector,Exchange,Website,Zip
AAPL,AAPL,Apple Inc.,Technology,NMS,https://www.apple.com,95014
MSFT,MSFT,Microsoft Corporation,Technology,NMS,https://www.microsoft.com,98052-6399
GOOGL,GOOGL,Alphabet Inc.,Communication Services,NMS,https://www.abc.xyz,94043
SAR,SAR,Saratoga Investment Corp New,Financial Services,NYQ,https://www.saratogainvestmentcorp.com,10022
AMZN,AMZN,"Amazon.com, Inc.",Consumer Cyclical,NMS,https://www.amazon.com,98109-5210
TSLA,TSLA,"Tesla, Inc.",Consumer Cyclical,NMS,https://www.tesla.com,78725
FB,FB,"Meta Platforms, Inc.",Communication Services,NMS,https://investor.fb.com,94025
NVDA,NVDA,NVIDIA Corporation,Technology,NMS,https://www.nvidia.com,95051
TSM,TSM,Taiwan Semiconductor Manufactur,Technology,NYQ,https://www.tsmc.com,300-78


In [12]:
def create_company_table(df):
    """
    Creating a table with all the information of the Companies
    Parameter Passed: company_info dataframe that is created.
    The reason why i am passing the entire table is because this information will not change, so it's best to create the table once
    and then just populate the table once and for all.
    """
    df.to_sql('Company_Information', engine, if_exists='replace', index = False)

    #Creating company_symbol as the primary key
    query = """ALTER TABLE "Company_Information" ADD PRIMARY KEY ("Company_Symbol");"""
    engine.execute(query)
    

    return 'Company Information Table Created'

create_company_table(company_info)

'Company Information Table Created'

In [13]:
#Getting historical prices (securities info) for one company and formatting the table a little bit.
price_history = yf.Ticker('AAPL').history(period='5y', interval='1d')
price_history = price_history.reset_index()
price_history['Company_Symbol'] = 'AAPL'
price_history

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Company_Symbol
0,2017-03-06,32.896886,32.991304,32.715138,32.889805,87000000,0.0,0.0,AAPL
1,2017-03-07,32.823717,33.040873,32.759985,32.932297,69785200,0.0,0.0,AAPL
2,2017-03-08,32.797752,32.998388,32.767070,32.809555,74828800,0.0,0.0,AAPL
3,2017-03-09,32.748186,32.759985,32.349278,32.734020,88623600,0.0,0.0,AAPL
4,2017-03-10,32.868570,32.894535,32.724586,32.842606,78451200,0.0,0.0,AAPL
...,...,...,...,...,...,...,...,...,...
1255,2022-02-28,163.059998,165.419998,162.429993,165.119995,94869100,0.0,0.0,AAPL
1256,2022-03-01,164.699997,166.600006,161.970001,163.199997,83474400,0.0,0.0,AAPL
1257,2022-03-02,164.389999,167.360001,162.949997,166.559998,79724800,0.0,0.0,AAPL
1258,2022-03-03,168.470001,168.910004,165.550003,166.229996,76335600,0.0,0.0,AAPL


In [14]:
def create_prices_table(ticker):

    """
    Create a daily prices table in the database. 
    This function essentially takes in a company symbol as the input, generates a price_history dataframe and then builds a table of daily prices
    using that dataframe. 
    So, we don't have to manually create the columns.
    A Query is also being run which makes the company symbol and date as the primary keys.
    """

    price_history = yf.Ticker('{}'.format(ticker)).history(period='5y', interval='1d')

    #some formatting
    price_history = price_history.reset_index()
    price_history['symbol'] = '{}'.format(ticker)
    price_history =price_history.drop(['Stock Splits'], axis=1)

    price_history.to_sql('daily_prices', engine, if_exists='replace', index=False)

    #creating company symbol as the primary key
    query = """ALTER TABLE daily_prices ADD PRIMARY KEY ("symbol", "Date");"""
    engine.execute(query)

    
    return 'Daily prices table created'

create_prices_table('NVDA')

'Daily prices table created'

In [15]:
def insert_historical_prices(ticker):

    """
    Inserts the historical prices of all the top 100 companies into the db.
    The function takes input as a company symbol, generates a price_history dataframe, does some formatting and then inserts the rows of that 
    dataframe into the daily_prices table.
    """
    price_history = yf.Ticker('{}'.format(ticker)).history(period='5y', interval='1d')

    price_history = price_history.fillna(0)

    #some formatting
    price_history = price_history.reset_index()
    price_history['symbol'] = '{}'.format(ticker)
    price_history = price_history.drop(['Stock Splits'], axis=1)

    insert_init = """INSERT INTO daily_prices (symbol, "Date", "Open", "High", "Low", "Close", "Volume", "Dividends") VALUES"""

    values = ",".join(["""('{}','{}','{}', '{}', '{}', '{}', '{}', '{}')""".format(
        row.symbol,
        row.Date,
        row.Open,
        row.High,
        row.Low,
        row.Close,
        row.Volume,
        row.Dividends
    ) for date, row in price_history.iterrows()])

    # insert_end = """ON CONFLICT (symbol, "Date") DO UPDATE
    #             SET
    #             Open = EXCLUDED.Open,
    #             High = EXCLUDED.High,
    #             Low = EXCLUDED.Low,
    #             Close = EXCLUDED.Close,
    #             Volume = EXCLUDED.Volume,
    #             Dividends = EXCLUDED.Dividends;
    #             """

    query = insert_init + values

    engine.execute(query)

In [16]:
#Adding historical prices of 8 companies.
top_companies = ['AAPL','MSFT','GOOGL','SAR','AMZN','TSLA','FB','TSM']
for ticker in top_companies:
  insert_historical_prices(ticker)
  print("{} Historical Data added".format(ticker))

AAPL Historical Data added
MSFT Historical Data added
GOOGL Historical Data added
SAR Historical Data added
AMZN Historical Data added
TSLA Historical Data added
FB Historical Data added
TSM Historical Data added


In [29]:
from datetime import date
from datetime import timedelta

today = date.today()
print(today)

yesterday = today - timedelta(days=1)
print(yesterday)

2022-03-06
2022-03-05


In [30]:
    price_history = yf.Ticker('AAPL').history(start = '{}'.format(yesterday), end = '{}'.format(today), period='5y', interval='1d')
    price_history

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-03-04,164.490005,165.550003,162.110001,163.169998,83819592,0,0
